In [1]:
%matplotlib inline
import math
import ipywidgets as widgets
from ipywidgets import HBox, VBox
from ipywidgets import interact
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display

In [2]:
wb = widgets.FloatSlider(value=0.3,min=0.15,max=0.6,step=0.025,description='bw (m):',readout_format='.3f')
wd = widgets.FloatSlider(value=0.5,min=0.20,max=1.0,step=0.025,description='d (m):',readout_format='.3f')
wcott = widgets.FloatSlider(value=2.5,min=1.0,max=2.5,step=0.1,description='cot(theta):')
wfck2 = widgets.Dropdown(
    options=[('C20/25', 20.0), ('C25/30', 25.0), ('C30/37', 30.0), ('C35/45', 35.0), ('C40/50', 40.0), ('C45/55', 45.0), ('C50/60', 50.0)],
    value=20,
    description='Concrete:',
)
wfyk2 = widgets.Dropdown(
    options=[('A400', 400.0), ('A500', 500.0)],
    value=400,
    description='Reinforcement:',
)
wved = widgets.FloatSlider(value=100,min=1,max=1000,step=1,description='Ved (kN):',readout_format='.0f')
wted = widgets.FloatSlider(value=100,min=1,max=1000,step=1,description='Ted (kN):',readout_format='.0f')

In [3]:
def shear_strengh(bw: float, d: float, fck: float, g_c: float, fyk: float, g_s: float, cott: float, asw_s: float, alpha: float) -> float:
    z = 0.9 * d
    vrd_s = asw_s * z * fyk / g_s * cott * 1000.0
    niu = 0.6*(1.0-fck/250)
    vrd_max = bw * z * niu * fck / g_c * 100.0 / (cott + 1.0/cott)
    return max(vrd_s, vrd_max)


def shear_design(bw: float, z: float, fck: float, g_c: float, fyk: float, g_s: float, cott: float, ved: float, alpha: float) -> float:
    niu = 0.6*(1.0-fck/250)
    vrd_max = bw * z * niu * fck / g_c * 1000.0 / (cott + 1.0/cott)

    asw_s = ved / z / fyk * g_s / cott / 1000.0 if vrd_max >= ved else math.nan
    return asw_s, vrd_max


def torsion_design(tef: float, Aek: float, uek: float, z: float, fck: float, g_c: float, fyk: float, g_s: float, cott: float, ted: float, alpha: float) -> float:
    niu = 0.6*(1.0-fck/250)
    trd_max = bw * z * niu * fck / g_c * 1000.0 / (cott + 1.0/cott)

    asw_s = ved / z / fyk * g_s / cott / 1000.0 if vrd_max >= ved else math.nan
    return asw_s, trd_max

In [4]:
@interact(bw=wb, d=wd, ved=wved, fck=wfck2, fyk=wfyk2, cott=wcott)
def shear(bw, d, fck, fyk, ved, cott):
    asw_s, vrd = shear_design(bw, d, fck, 1.5, fyk, 1.15, cott, ved, 90)
    print(f'{asw_s=:.6f} cm2     {vrd=:.0f} kN')

interactive(children=(FloatSlider(value=0.3, description='bw (m):', max=0.6, min=0.15, readout_format='.3f', s…